In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch

In [4]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact_manual
import ipywidgets as widgets
import torch
import sys
import os
sys.path.append(os.path.expanduser('~/Desktop/Basset_E/'))
from sample.utils.load_model import *
import numpy as np
from Interpretation.utils import *
import json
PDBX = os.environ['PDBX_UBC']

# Data loading

### Load Jaspar Name -> Matrix ID

In [5]:
with open(PDBX+'data/vertebrates.profiles.json') as handle:
    matrix_2_name = json.loads(handle.read())
matrix_2_name = {k.upper(): v for k, v in matrix_2_name.items()}
name_2_matrix = {v: k for k, v in matrix_2_name.items()}

### Load list motif Sasha

In [6]:
with open(PDBX+'data/sasha_ATAC/motifs_sasha.json') as handle:
    name_2_string = json.loads(handle.read())
name_2_string = {k.upper() : v for k,v in name_2_string.items()}

In [ ]:
# Load Model
# Load Model ( please use the same as the one for motif generation)
t = torch.load(os.path.expanduser(PDBX+'Models/Basset_2019-11-04_19_23_35/model_loss_0.68.tar'))
m, _ = load_model_E()
m.load_state_dict(t['model_state_dict'])
m.eval();
activation = lambda x : 8*torch.sigmoid(x)+1
# Load Data
x = np.load(os.path.expanduser(PDBX+"/data/perm_one_hot_seqs.npy")) #load permuted sequences
x = x.astype(np.float32)

### Load Global Counts

In [12]:
run = 'Run_28_11_I1'
generated = pd.read_csv(PDBX+'data/Runs_GA/{}/global_count_motifs_sasha.csv'.format(run),index_col=0).fillna(0)

# Analysis

## JASPAR Motifs

In [13]:
@interact_manual
def plot_motifs_for_cell_type(gtask=generated.index.sort_values(), sort = False, n_disp = widgets.IntSlider(min=1, max=len(generated.columns), step=1, value=10)) :
    fig = go.Figure()
    if sort : 
        index_disp = generated.loc[gtask].sort_values(ascending=False)[0:n_disp].index
        print('Most frequent motif generation : '+' '.join(index_disp))
    else : 
        index_disp =  generated.columns
    fig.add_trace(go.Bar(x=index_disp, y=generated.loc[gtask,index_disp],name='generated'))
    fig.show()

interactive(children=(Dropdown(description='gtask', options=('filter1', 'filter10', 'filter102', 'filter105', …

In [14]:
@interact_manual
def plot_motifs_for_cell_type(motif=widgets.Combobox(options=list(generated.columns),ensure_option=True,disabled=False)) :
    if name_2_matrix.get(motif) :
        fig = go.Figure()
        fig.add_trace(go.Bar(x=generated.index, y=generated.loc[:,motif], name='generated'))
        fig.show()
        h,b, _ = simulate(m, activation, x,matrix_id='MA0014.1')
        b.show()
        h.show()

interactive(children=(Combobox(value='', description='motif', ensure_option=True, options=('B', 'innate.lym', …